# 037. 토크나이저 처음부터 학습 시키기

In [1]:
# KoNLPy(한국어 형태소 분석기 패키지) 설치
!pip install -q KoNLPy

# SentencePiece(서브워드 토크나이저 도구)를 최신 버전으로 업그레이드 및 설치
!pip install -U -q sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 9.0 MB/s eta 0:00:00


In [2]:
sentences_E = [
    'I love my dog',
    'I love my cat',
    'You love my dog!',
    'I was born in Korea and graduaged University in USA.',
]

sentences_K = [
    "코로나가 심하다",
    "코비드-19가 심하다",
    '아버지가방에들어가신다',
    '아버지가 방에 들어가신다',
    '너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다'
]

# 1. Keras 기본 Tokenizer - rule-based
- 공백 또는 구둣점으로 분리  
- 영어 단어별로 띄어쓰기가 철저히 지켜지는 언어

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 빈도수 상위 100개의 단어로 구성된 Tokenizer 객체 생성 (OOV(Out-Of-Vocabulary) 토큰 설정)
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')

# 주어진 문장 리스트에 대해 토크나이저 학습 수행 (단어 인덱스 구축)
tokenizer.fit_on_texts(sentences_E)

# 구축된 단어 인덱스 사전 가져오기
word_index = tokenizer.word_index

# 단어 인덱스 사전 출력
print(word_index)

{'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'in': 6, 'cat': 7, 'you': 8, 'was': 9, 'born': 10, 'korea': 11, 'and': 12, 'graduaged': 13, 'university': 14, 'usa': 15}


Keras의 rule base tokenizer로 한글을 tokenize

In [4]:
# 빈도수 상위 100개의 단어로 구성된 Tokenizer 객체 생성 (OOV(Out-Of-Vocabulary) 토큰 설정)
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')

# 주어진 한글 문장 리스트에 대해 토크나이저 학습 수행 (단어 인덱스 구축)
tokenizer.fit_on_texts(sentences_K)

# 구축된 단어 인덱스 사전 가져오기
vocabulary_keras_korean = tokenizer.word_index

# 단어 인덱스 사전 출력
print(vocabulary_keras_korean)

{'<OOV>': 1, '심하다': 2, '코로나가': 3, '코비드': 4, '19가': 5, '아버지가방에들어가신다': 6, '아버지가': 7, '방에': 8, '들어가신다': 9, '너무너무너무는': 10, '나카무라세이코가': 11, '불러': 12, '크게': 13, '히트한': 14, '노래입니다': 15}


# 2. 단어 사전 기반 한국어 tokenizer 사용

In [5]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 형태소 분석 결과를 저장할 리스트 초기화
temp_X = []

# 주어진 한글 문장 리스트의 각 문장에 대해 반복
for sent in sentences_K:
    # 문장을 형태소 분석하여 결과를 리스트에 추가
    temp_X.append(okt.morphs(sent))
    # 형태소 분석 결과 출력
    print(okt.morphs(sent))

['코로나', '가', '심하다']
['코', '비드', '-', '19', '가', '심하다']
['아버지', '가방', '에', '들어가신다']
['아버지', '가', '방', '에', '들어가신다']
['너무', '너무', '너', '무', '는', '나카무라', '세이', '코', '가', '불러', '크게', '히트', '한', '노래', '입니다']


사전 기반 tokenize 후 Keras tokenizer 로 vocabulary 생성

In [6]:
# 빈도수 상위 100개의 단어로 구성된 Tokenizer 객체 생성 (OOV(Out-Of-Vocabulary) 토큰 설정)
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')

# 형태소 분석된 문장 리스트에 대해 토크나이저 학습 수행 (단어 인덱스 구축)
tokenizer.fit_on_texts(temp_X)

# 구축된 단어 인덱스 사전 가져오기
vocabulary_okt_keras = tokenizer.word_index

# 단어 인덱스 사전 출력
print(vocabulary_okt_keras)

{'<OOV>': 1, '가': 2, '심하다': 3, '코': 4, '아버지': 5, '에': 6, '들어가신다': 7, '너무': 8, '코로나': 9, '비드': 10, '-': 11, '19': 12, '가방': 13, '방': 14, '너': 15, '무': 16, '는': 17, '나카무라': 18, '세이': 19, '불러': 20, '크게': 21, '히트': 22, '한': 23, '노래': 24, '입니다': 25}


두 vocabulary 의 차이 비교

In [7]:
print(vocabulary_keras_korean)
print(vocabulary_okt_keras)

{'<OOV>': 1, '심하다': 2, '코로나가': 3, '코비드': 4, '19가': 5, '아버지가방에들어가신다': 6, '아버지가': 7, '방에': 8, '들어가신다': 9, '너무너무너무는': 10, '나카무라세이코가': 11, '불러': 12, '크게': 13, '히트한': 14, '노래입니다': 15}
{'<OOV>': 1, '가': 2, '심하다': 3, '코': 4, '아버지': 5, '에': 6, '들어가신다': 7, '너무': 8, '코로나': 9, '비드': 10, '-': 11, '19': 12, '가방': 13, '방': 14, '너': 15, '무': 16, '는': 17, '나카무라': 18, '세이': 19, '불러': 20, '크게': 21, '히트': 22, '한': 23, '노래': 24, '입니다': 25}


### 단, Okt 사전에 미등록된 단어의 경우 정확한 tokenizing 이 안된다.

In [8]:
# 주어진 문장을 형태소 분석하여 품사 태깅 수행
okt.pos('너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다')

[('너무', 'Adverb'),
 ('너무', 'Adverb'),
 ('너', 'Modifier'),
 ('무', 'Noun'),
 ('는', 'Josa'),
 ('나카무라', 'Noun'),
 ('세이', 'Noun'),
 ('코', 'Noun'),
 ('가', 'Josa'),
 ('불러', 'Verb'),
 ('크게', 'Noun'),
 ('히트', 'Noun'),
 ('한', 'Josa'),
 ('노래', 'Noun'),
 ('입니다', 'Adjective')]

예를 들어 `너무너무너무`와 `나카무라세이코`는 하나의 단어이지만, okt 사전에 등록되어 있지 않아 여러 개의 복합단어로 나뉘어집니다. 이러한 문제를 해결하기 위하여 형태소 분석기와 품사 판별기들은 사용자 사전 추가 기능을 제공합니다. 사용자 사전을 추가하여 모델의 vocabulary 를 풍부하게 만드는 것은 사용자의 몫입니다.

1. okt 공식 문서를 참고해서 사용사 사전을 추가.
2. okt를 패키징하고, konlpy에서 사용할 수 있도록 konlpy/java 경로에 jar 파일을 복사.
3. 기존에 참고하고 있던 okt.jar 대신 새로운 okt.jar를 사용하도록 설정.
4. konlpy 소스 경로를 import 해서 형태소 분석.

# 3. Google SentencePiece Tokenizer

- NAVER Movie rating data 를 이용한 sentencepiece tokenizer training

In [9]:
import tensorflow as tf
import pandas as pd
import sentencepiece as spm

DATA_TRAIN_PATH = tf.keras.utils.get_file("ratings_train.txt",
        "https://github.com/ironmanciti/infran_NLP/raw/main/data/naver_movie/ratings_train.txt")

14628807/14628807 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


- pandas.read_csv에서 quoting = 3으로 설정해주면 인용구(따옴표)를 무시

In [10]:
train_data = pd.read_csv(DATA_TRAIN_PATH, sep='\t', quoting=3)

print(train_data.shape)
train_data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train_data.isnull().sum()

,0
id,0
document,5
label,0


In [12]:
train_data.dropna(inplace=True)

train_data.shape

(149995, 3)

## 학습을 위해 text 를 따로 저장

In [13]:
# 'nsmc.txt' 파일을 쓰기 모드로 열기 (UTF-8 인코딩 사용)
with open('./nsmc.txt', 'w', encoding='utf-8') as f:
    # 훈련 데이터의 'document' 열에 있는 각 문장에 대해 반복
    for line in train_data.document.values:
        try:
            # 문장을 파일에 쓰고 새로운 줄 추가
            f.write(line + '\n')
        except:
            # 쓰기 오류 발생 시 오류 메시지와 해당 문장 출력
            print("write error ---> ", line)

In [14]:
#write 가 잘 되었는지 확인
with open('./nsmc.txt', 'r', encoding='utf-8') as f:
    nsmc_txt = f.read().split('\n')

print(len(nsmc_txt))
print(nsmc_txt[0])

149996
아 더빙.. 진짜 짜증나네요 목소리


In [15]:
# 입력 파일 경로 설정
input_file = 'nsmc.txt'

# 어휘 사전의 최대 크기 설정
vocab_size = 30000

# 모델 파일의 접두사 설정
prefix = 'nsmc'

# 명령어 템플릿 정의
templates = '--input={} --model_prefix={} --vocab_size={}'

# 템플릿에 변수 값을 포맷하여 명령어 문자열 생성
cmd = templates.format(input_file, prefix, vocab_size)

# 생성된 명령어 출력
print(cmd)

--input=nsmc.txt --model_prefix=nsmc --vocab_size=30000


### sentencepiece tokenizer training

In [16]:
# SentencePieceTrainer를 사용하여 SentencePiece 모델 학습
spm.SentencePieceTrainer.Train(cmd)

In [17]:
# SentencePieceProcessor 객체 생성
sp = spm.SentencePieceProcessor()

# 학습된 SentencePiece 모델 로드
sp.Load('{}.model'.format(prefix))

True

In [18]:
# 훈련 데이터의 'document' 열에 있는 첫 세 개의 문장에 대해 반복
for t in train_data.document.values[:3]:
    # 원본 문장 출력
    print(t)
    # 문장을 SentencePiece 모델을 사용하여 토큰화하여 출력
    print(sp.encode_as_pieces(t))
    # 문장을 SentencePiece 모델을 사용하여 인덱스 시퀀스로 변환하여 출력
    print(sp.encode_as_ids(t), '\n')

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더빙', '..', '▁진짜', '▁짜증나네요', '▁목소리']
[53, 751, 5, 25, 15853, 1405] 

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁흠', '...', '포스터보고', '▁초딩영화', '줄', '....', '오버', '연기', '조차', '▁가볍지', '▁않', '구나']
[1239, 6, 12536, 18315, 396, 47, 17886, 395, 1134, 6404, 1063, 423] 

너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그래서', '보는것', '을', '추천', '한다']
[18, 611, 21195, 640, 2752, 11171, 14, 2315, 298] 



In [19]:
# 한글 문장 리스트(sentences_K)에 있는 각 문장에 대해 반복
for line in sentences_K:
    # 문장을 SentencePiece 모델을 사용하여 토큰화
    pieces = sp.encode_as_pieces(line)
    # 문장을 SentencePiece 모델을 사용하여 인덱스 시퀀스로 변환
    ids = sp.encode_as_ids(line)
    # 원본 문장 출력
    print(line)
    # 토큰화된 결과 출력
    print(pieces)
    # 인덱스 시퀀스 출력
    print(ids)
    # 각 문장 사이에 줄 바꿈 추가
    print()

코로나가 심하다
['▁코', '로', '나', '가', '▁심하다']
[1482, 29, 33, 13, 5371]

코비드-19가 심하다
['▁코', '비', '드', '-', '19', '가', '▁심하다']
[1482, 334, 266, 287, 3859, 13, 5371]

아버지가방에들어가신다
['▁아버지가', '방', '에', '들어가', '신', '다']
[6158, 627, 16, 13026, 272, 23]

아버지가 방에 들어가신다
['▁아버지가', '▁방', '에', '▁들어가', '신', '다']
[6158, 1673, 16, 3872, 272, 23]

너무너무너무는 나카무라세이코가 불러 크게 히트한 노래입니다
['▁너무너무너무', '는', '▁나카', '무라', '세', '이', '코가', '▁불러', '▁크게', '▁히트', '한', '▁노래', '입니다']
[14213, 12, 17034, 10019, 247, 10, 12900, 3403, 1856, 12030, 30, 763, 222]

